# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
from dotenv import load_dotenv, find_dotenv
from utils import get_openai_api_key
_ = load_dotenv(override=True)
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

[autogen.oai.client: 01-18 16:32:13] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


/Users/ravikumartummalapenta/Documents/github/rktummalapenta/AI-Agents-in-LangGraph/.agents_env/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a platform revolutionizing how we interact with technology. From cutting-edge courses to practical applications, DeepLearning.AI offers a gateway to mastering deep learning. Whether you're a beginner or an expert, the platform provides the tools and resources needed to thrive in this rapidly evolving field. Join a community of like-minded individuals and embark on a journey towards innovation and discovery. Explore the endless possibilities with DeepLearning.AI and unlock the potential of AI today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 01-18 16:32:23] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [9]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a platform revolutionizing how we interact with technology. From cutting-edge courses to practical applications, DeepLearning.AI offers a gateway to mastering deep learning. Whether you're a beginner or an expert, the platform provides the tools and resources needed to thrive in this rapidly evolving field. Join a community of like-minded individuals and embark on a journey towards innovation and discovery. Explore the endless possibilities with DeepLearning.AI and unlock the potential of AI today!

--------------------------------------------------------------------------------
Critic (to Writer):

This blogpost eff

## Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


[autogen.oai.client: 01-18 16:44:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 01-18 16:44:15] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 01-18 16:44:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 01-18 16:44:17] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Orchestrate the nested chats to solve the task

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI, a platform revolutionizing how we interact with technology. From cutting-edge courses to practical applications, DeepLearning.AI offers a gateway to mastering deep learning. Whether you're a beginner or an expert, the platform provides the tools and resources needed to thrive in this rapidly evolving field. Join a community of like-minded individuals and embark on a journey towards innovation and discovery. Explore the endless possibilities with DeepLearning.AI and unlock the potential of AI today!

--------------------------------------------------------------------------------

*************************************

## Get the summary

In [18]:
print(res.summary)

Title: Navigating DeepLearning.AI: Unleashing AI's Potential Responsibly

Embark on an AI journey with DeepLearning.AI's innovative platform reshaping tech interactions. Discover top-notch courses and applications, catering to all skill levels. Engage with a vibrant community for continuous learning and growth. Dive into the AI realm responsibly, ensuring transparency and accuracy in content. Embrace SEO best practices with optimized keywords, meta descriptions, and internal links. Uphold compliance with regulations and clarify user data handling for privacy. Unleash AI's potential ethically and confidently with DeepLearning.AI!
